In [ ]:
print("hii")

hii


# French To English Translator USING **LSTM**

## important library 

In [ ]:
from tensorflow.keras.layers import Input, LSTM
import numpy as np 
patch_size = 64 # Batch size for training.
epochs = 100 # Number of epochs to train for.
latent_dim = 256 # Latent dimensionality of the
num_samples = 1000 # Number of samples to train
# Path to the data txt file on disk.


## Data Set

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np 
batch_size = 64 # Batch size for training.
epochs = 100 # Number of epochs to train for.
latent_dim = 256 # Latent dimensionality of the encoding space.
num_samples = 10000 # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "/content/drive/MyDrive/Colab Notebooks/fra.txt"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Test Preprocessing


In [ ]:
#, Vectorize the data.
input_texts =[]
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text, _ = line.split('\t')
  # We use “tab” as the “start sequence” character
  # for the targets, and "\n" as “end sequence” character.
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
      if char not in target_characters:
        target_characters.add(char)

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)

num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


In [ ]:
print(len(input_characters))

71


In [ ]:
input_token_index=dict(
    [(char,i) for i , char in enumerate(input_characters)])
target_token_index=dict(
    [(char,i) for i, char in enumerate(target_characters)]
)

In [ ]:
encoder_input_data = np.zeros(
(len(input_texts), max_encoder_seq_length, num_encoder_tokens),
dtype='float32')

decoder_input_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens),
dtype='float32')

decoder_target_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens),
dtype='float32') 


In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]]=1.
  encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
  for t,char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.
  decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
  decoder_target_data[i, t:, target_token_index[' ']] = 1.

 
    


#MODEL

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#TRAINING

In [ ]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 57s 429ms/step - loss: 1.1290 - val_loss: 1.0101
Epoch 2/100
125/125 [==============================] - 48s 388ms/step - loss: 0.8164 - val_loss: 0.8322
Epoch 3/100
125/125 [==============================] - 49s 394ms/step - loss: 0.6535 - val_loss: 0.6947
Epoch 4/100
125/125 [==============================] - 51s 406ms/step - loss: 0.5686 - val_loss: 0.6298
Epoch 5/100
125/125 [==============================] - 59s 472ms/step - loss: 0.5219 - val_loss: 0.5831
Epoch 6/100
125/125 [==============================] - 54s 431ms/step - loss: 0.4877 - val_loss: 0.5674
Epoch 7/100
125/125 [==============================] - 52s 414ms/step - loss: 0.4613 - val_loss: 0.5347
Epoch 8/100
125/125 [==============================] - 48s 385ms/step - loss: 0.4383 - val_loss: 0.5151
Epoch 9/100
125/125 [==============================] - 51s 412ms/step - loss: 0.4184 - val_loss: 0.5003
Epoch 10/100
125/125 [==============================] - 48s 385m

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

TESTING

In [30]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

##LINK FOR DATASET AND EXPLATION OF CODE 

https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [49]:
"""for seq_index in range(100):
  # Take one sequence (part of the training set)
  # for trying out decoding.
  input_seq = (encoder_input_data[seq_index: seq_index+1]).astype(int)
  #decoded_sentence= decode_sequence(input_seq)
  #print("hello")
  #print('Input sentence:',input_texts[input_seq.astype(int)])
  print('Input sentence:',input_texts[input_seq])
  print('Decoded sentence:', decode_sequence(input_seq))"""
  

 


'for seq_index in range(100):\n  # Take one sequence (part of the training set)\n  # for trying out decoding.\n  input_seq = (encoder_input_data[seq_index: seq_index+1]).astype(int)\n  #decoded_sentence= decode_sequence(input_seq)\n  #print("hello")\n  #print(\'Input sentence:\',input_texts[input_seq.astype(int)])\n  print(\'Input sentence:\',input_texts[input_seq])\n  print(\'Decoded sentence:\', decode_sequence(input_seq))'

In [40]:
#print('Input sentence:',input_texts[seq_index])